# Lab | Hypothesis Testing

**Objective**

Welcome to the Hypothesis Testing Lab, where we embark on an enlightening journey through the realm of statistical decision-making! In this laboratory, we delve into various scenarios, applying the powerful tools of hypothesis testing to scrutinize and interpret data.

From testing the mean of a single sample (One Sample T-Test), to investigating differences between independent groups (Two Sample T-Test), and exploring relationships within dependent samples (Paired Sample T-Test), our exploration knows no bounds. Furthermore, we'll venture into the realm of Analysis of Variance (ANOVA), unraveling the complexities of comparing means across multiple groups.

So, grab your statistical tools, prepare your hypotheses, and let's embark on this fascinating journey of exploration and discovery in the world of hypothesis testing!

**Challenge 1**

In this challenge, we will be working with pokemon data. The data can be found here:

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv

In [1]:
#libraries
import pandas as pd
import scipy.stats as st
import numpy as np



In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv")
df.head()

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


- We posit that Pokemons of type Dragon have, on average, more HP stats than Grass. Choose the propper test and, with 5% significance, comment your findings.

In [1]:
#code here
#hypothesis
# H0 type_dragon_hp > type_grass_hp
# H1 type_dragon_hp <= type_grass_hp

In [50]:
df_dragon=df[(df["Type 1"]=="Dragon")]["HP"]
df_grass=df[(df["Type 1"]=="Grass")]["HP"]

alpha = 0.05

resulat=st.ttest_ind(df_grass,df_dragon, alternative='less')
print(resulat)
resulat2=st.ttest_ind(df_dragon,df_grass ,alternative='greater')
print(resulat2)


TtestResult(statistic=-3.590444254130357, pvalue=0.0002567969150153481, df=100.0)
TtestResult(statistic=3.590444254130357, pvalue=0.0002567969150153481, df=100.0)


Dragons ont plus de PV.

- We posit that Legendary Pokemons have different stats (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed) when comparing with Non-Legendary. Choose the propper test and, with 5% significance, comment your findings.


In [47]:
#code here
df_legend=df[(df['Legendary']==1)]
df_non_legend=df[(df['Legendary']==0)]

caract = list(df.columns)
spec=caract[3:9]
print(spec)
for i in spec:
        comparaison=st.ttest_ind(df_legend[i],df_non_legend[i], equal_var=True)
        print(i)
        print(comparaison)

['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
HP
TtestResult(statistic=8.036124405043928, pvalue=3.3306476848461913e-15, df=798.0)
Attack
TtestResult(statistic=10.397321023700622, pvalue=7.827253003205333e-24, df=798.0)
Defense
TtestResult(statistic=7.181240122992339, pvalue=1.5842226094427255e-12, df=798.0)
Sp. Atk
TtestResult(statistic=14.191406210846289, pvalue=6.314915770427265e-41, df=798.0)
Sp. Def
TtestResult(statistic=11.03775106120522, pvalue=1.8439809580409597e-26, df=798.0)
Speed
TtestResult(statistic=9.765234331931898, pvalue=2.3540754436898437e-21, df=798.0)


**Challenge 2**

In this challenge, we will be working with california-housing data. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv

In [58]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv")
df.head()



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


**We posit that houses close to either a school or a hospital are more expensive.**

- School coordinates (-118, 37)
- Hospital coordinates (-122, 34)

We consider a house (neighborhood) to be close to a school or hospital if the distance is lower than 0.50.

Hint:
- Write a function to calculate euclidean distance from each house (neighborhood) to the school and to the hospital.
- Divide your dataset into houses close and far from either a hospital or school.
- Choose the propper test and, with 5% significance, comment your findings.
 

In [51]:
def dist_school_hospital (longitude, latitude):
    distance_school= np.sqrt((-118-longitude)**2+(37-latitude)**2)
    distance_hospital= np.sqrt((-112-longitude)**2+(34-latitude)**2)
    return min(distance_school,distance_hospital)

In [63]:
def dist_school_hospital (longitude, latitude):
    distance_school= np.sqrt((-118-longitude)**2+(37-latitude)**2)
    distance_hospital= np.sqrt((-112-longitude)**2+(34-latitude)**2)
    return min(distance_school,distance_hospital)
df['dist']=df.apply(lambda row : dist_school_hospital(row['longitude'],row['latitude']),axis=1)
df.head(10)



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,dist
6904,-118.31,36.94,35.0,2563.0,530.0,861.0,371.0,2.3250,80600.0,0.315753
6776,-118.30,37.17,22.0,3480.0,673.0,1541.0,636.0,2.7500,94500.0,0.344819
4523,-118.05,36.64,34.0,2090.0,478.0,896.0,426.0,2.0357,74200.0,0.363456
5596,-118.18,37.35,16.0,3806.0,794.0,1501.0,714.0,2.1212,108300.0,0.393573
5597,-118.18,36.63,23.0,2311.0,487.0,1019.0,384.0,2.2574,104700.0,0.411461
...,...,...,...,...,...,...,...,...,...,...
16987,-124.21,41.77,17.0,3461.0,722.0,1947.0,647.0,2.5795,68400.0,7.830517
16991,-124.23,41.75,11.0,3159.0,616.0,1343.0,479.0,2.4805,73200.0,7.834245
16939,-124.14,41.95,21.0,2696.0,578.0,1208.0,494.0,2.2750,122400.0,7.886831
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0,7.920227


In [68]:
df_close=df[df['dist']<0.5]['median_house_value']

df_far=df[(df['dist']>=0.5)]['median_house_value']

resulat=st.ttest_ind(df_close,df_far, alternative='less')

print(resulat)

TtestResult(statistic=-2.2146147257665834, pvalue=0.013399866535564343, df=16998.0)


In [71]:
df_close.mean(),df_far.mean()
df_close

(92460.0, 207334.6990291262)

4523     74200.0
5596    108300.0
5597    104700.0
6776     94500.0
6904     80600.0
Name: median_house_value, dtype: float64